In [1]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
load(file=file.path(saveDir, "tseCons.RData"))
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(beaveR))

In [2]:
tree <- rowTree(tse)
l <- length(tree$tip)

In [3]:
txpSf <- computeSizeFactors(tse)
txpSf <- metadata(txpSf)[["sf"]]
geneSf <- computeSizeFactors(tse, type = "gene")
geneSf <- metadata(geneSf)[["sf"]]

useHub=TRUE: checking for TxDb via 'AnnotationHub'

snapshotDate(): 2021-05-18

found matching TxDb via 'AnnotationHub'

loading from cache

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

obtaining transcript-to-gene mapping from database

generating gene ranges

summarizing abundance

summarizing counts

summarizing length

summarizing inferential replicates



In [26]:
gse <- summarizeToGene(tse[1:l,])

loading existing TxDb created: 2023-02-03 16:37:13

obtaining transcript-to-gene mapping from database

loading existing gene ranges created: 2023-02-03 16:37:21

summarizing abundance

summarizing counts

summarizing length

summarizing inferential replicates



In [4]:
head(txpSf)
round(matrixStats::colSds(txpSf),3)
round(colMeans(txpSf),3)

1.061357,1.058792,1.089445,1.068088,1.081509,1.079970,1.081856,1.069232,1.068193,1.055898,1.035655,1.058276
1.061038,1.060825,1.090349,1.071358,1.076601,1.074738,1.086419,1.069260,1.070558,1.060018,1.035796,1.056605
1.061215,1.055809,1.093435,1.068338,1.081280,1.078589,1.083503,1.071053,1.067617,1.058252,1.035619,1.057153
1.064243,1.056570,1.089089,1.070159,1.079450,1.077500,1.080737,1.069639,1.066094,1.059219,1.037925,1.058461
1.060263,1.062008,1.088108,1.070858,1.079198,1.076538,1.082660,1.065982,1.071550,1.057016,1.037113,1.058506
1.061683,1.061159,1.090614,1.068914,1.081328,1.078889,1.079531,1.068017,1.068549,1.056580,1.038294,1.059078


[1] 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002

[1] 1.062 1.059 1.089 1.069 1.080 1.079 1.083 1.069 1.069 1.058 1.036 1.057

In [5]:
head(geneSf)
round(matrixStats::colSds(geneSf),3)
round(colMeans(geneSf),3)

1.0002974,0.9974998,1.028293,1.006715,1.030797,1.021908,1.022671,1.008912,1.009508,0.9980879,0.9815208,1.003293
0.9998755,1.0001648,1.030503,1.007665,1.027995,1.022486,1.024837,1.010945,1.009608,0.9982530,0.9816668,1.003027
0.9995685,0.9968015,1.029075,1.006174,1.028791,1.022906,1.024271,1.010946,1.008738,0.9976935,0.9801276,1.001972
1.0003836,0.9982278,1.028722,1.007049,1.029870,1.019133,1.021961,1.009640,1.009008,0.9978210,0.9833754,1.002528
0.9984739,0.9984468,1.028058,1.007541,1.030257,1.022527,1.025599,1.010558,1.010368,0.9983992,0.9816788,1.002531
1.0015917,1.0003767,1.027845,1.007971,1.032246,1.022804,1.023844,1.011168,1.010127,0.9955624,0.9825007,1.001840


[1] 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001

[1] 1.001 0.998 1.029 1.008 1.030 1.022 1.023 1.010 1.010 0.998 0.981 1.003

In [9]:
suppressPackageStartupMessages(library(DESeq2))
suppressWarnings(dds <- DESeqDataSet(tse[1:l,], ~1))

using counts and average transcript lengths from tximeta



In [19]:
sfTxpDes <- matrix(0, 100, 12)
for(i in seq(100)) {
    ir <- paste("infRep",i, sep="")
    sfTxpDes[i,] <- estimateSizeFactorsForMatrix(assays(dds)[[ir]])
}


In [23]:
round(matrixStats::colSds(sfTxpDes),3)
round(colMeans(sfTxpDes),3)

[1] 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002 0.002

[1] 1.043 1.066 1.125 1.060 1.167 1.152 1.039 1.063 1.120 1.054 1.160 1.145

In [31]:
ddsGene <- DESeqDataSet(gse, ~1)
sfGeneDes <- matrix(0, 100, 12)
for(i in seq(100)) {
    ir <- paste("infRep",i, sep="")
    sfGeneDes[i,] <- estimateSizeFactorsForMatrix(assays(ddsGene)[[ir]])
}


using counts and average transcript lengths from tximeta



In [32]:
round(matrixStats::colSds(sfGeneDes),3)
round(colMeans(sfGeneDes),3)

[1] 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001

[1] 0.943 0.973 1.041 0.961 1.104 1.080 0.939 0.971 1.041 0.954 1.094 1.075